In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv('/content/drive/My Drive/Hackathon/Janatahack Independence Day 2020 ML Hackathon/train.csv')
test = pd.read_csv('/content/drive/My Drive/Hackathon/Janatahack Independence Day 2020 ML Hackathon/test.csv')
samp = pd.read_csv('/content/drive/My Drive/Hackathon/Janatahack Independence Day 2020 ML Hackathon/sample_submission_UVKGLZE.csv')

In [3]:
train.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [4]:
test.head()

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [5]:
samp.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,0,0,0
1,20974,0,0,0,0,0,0
2,20975,0,0,0,0,0,0
3,20976,0,0,0,0,0,0
4,20977,0,0,0,0,0,0


In [6]:
print('Train shape:',train.shape)
print('Test shape:',test.shape)
print('Sample shape:',samp.shape)

Train shape: (20972, 9)
Test shape: (8989, 3)
Sample shape: (8989, 7)


In [7]:
train.isnull().sum()

ID                      0
TITLE                   0
ABSTRACT                0
Computer Science        0
Physics                 0
Mathematics             0
Statistics              0
Quantitative Biology    0
Quantitative Finance    0
dtype: int64

In [8]:
l = ['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']

for col in l:
  print(col,':\n',train[col].value_counts())

Computer Science :
 0    12378
1     8594
Name: Computer Science, dtype: int64
Physics :
 0    14959
1     6013
Name: Physics, dtype: int64
Mathematics :
 0    15354
1     5618
Name: Mathematics, dtype: int64
Statistics :
 0    15766
1     5206
Name: Statistics, dtype: int64
Quantitative Biology :
 0    20385
1      587
Name: Quantitative Biology, dtype: int64
Quantitative Finance :
 0    20723
1      249
Name: Quantitative Finance, dtype: int64


In [9]:
#CS :8594
#Phy :6013
#Math :5618
#Stats :5206
#QB :587
#QF :249

In [10]:
dic = {'CS' :8594,'Phy' :6013,'Math' :5618,'Stats' :5206,'QB' :587,'QF' :249}

values = dic.values()

total = sum(values)

new = [value * 100. / total for value in values]
print(new)

[32.71785891041992, 22.891841474092967, 21.388053451098337, 19.819545437240645, 2.234743213918605, 0.9479575132295276]


In [11]:
#from above we can say that the data is imbalanced for last 2 features

In [12]:
train['combined'] = train['TITLE']+' '+train['ABSTRACT']
test['combined'] = test['TITLE']+' '+test['ABSTRACT']

In [13]:
test = test.drop(['ID','TITLE','ABSTRACT'],axis=1)

In [14]:
X = train.loc[:,['combined']]
y = train.loc[:,l]

In [15]:
train_size = round(train.shape[0]*0.8)
test_size = train.shape[0] - train_size
print(train_size)
print(test_size)

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

16778
4194
(16778, 1) (4194, 1)
(16778, 6) (4194, 6)


In [16]:
y1 = np.array(y_train)
y2 = np.array(y_test)

In [17]:
y_test.reset_index(drop=True,inplace=True)
y_test.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,0,0,1,0,0,0
1,0,0,0,1,0,0
2,0,0,1,0,0,0
3,1,0,0,0,0,0
4,0,0,1,0,0,0


#TEXT PREPROCESSING:

In [18]:
#Removing Punctuations

X_train.replace('[^a-zA-Z]',' ', regex=True, inplace=True)
X_test.replace('[^a-zA-Z]',' ', regex=True, inplace=True)

test.replace('[^a-zA-Z]',' ', regex=True, inplace=True)

In [19]:
#Converting to lower case characters

for index in X_train.columns:
  X_train[index] = X_train[index].str.lower()

for index in X_test.columns:
  X_test[index] = X_test[index].str.lower()

for index in test.columns:
  test[index] = test[index].str.lower()

In [20]:
#Removing one letter words

X_train['combined'] = X_train['combined'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
X_test['combined'] = X_test['combined'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

test['combined'] = test['combined'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

In [21]:
#Removing multiple blank spaces

X_train = X_train.replace('\s+', ' ', regex=True)
X_test = X_test.replace('\s+', ' ', regex=True)

test = test.replace('\s+', ' ', regex=True)

In [22]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
stop_words = set(stopwords.words('english')) 
len(stop_words)
X_train['combined'] = X_train['combined'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
X_test['combined'] = X_test['combined'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

test['combined'] = test['combined'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [24]:
train_lines = []
for row in range(0,X_train.shape[0]):
  train_lines.append(' '.join(str(x) for x in X_train.iloc[row,:]))

test_lines = []
for row in range(0,X_test.shape[0]):
  test_lines.append(' '.join(str(x) for x in X_test.iloc[row,:]))

predtest_lines = []
for row in range(0,test.shape[0]):
  predtest_lines.append(' '.join(str(x) for x in test.iloc[row,:]))

In [25]:
train_lines

['reconstructing subject specific effect maps predictive models allow subject specific inference analyzing disease related alterations neuroimaging data given subject data inference made two levels global identifiying condition presence subject local detecting condition effect individual measurement extracted subject data global inference widely used local inference used form subject specific effect maps rarely used existing models often yield noisy detections composed dispersed isolated islands article propose reconstruction method named rsm improve subject specific detections predictive modeling approaches particular binary classifiers rsm specifically aims reduce noise due sampling error associated using finite sample examples train classifiers proposed method wrapper type algorithm used different binary classifiers diagnostic manner without information condition presence reconstruction posed maximum posteriori problem prior model whose parameters estimated training data classifier 

In [26]:
test_lines

['fundamental limits low rank matrix estimation non symmetric case consider high dimensional inference problem signal low rank matrix corrupted additive gaussian noise given probabilistic model low rank matrix compute limit large dimension setting mutual information signal observations well matrix minimum mean square error rank signal remains constant allows locate information theoretic threshold estimation problem critical value signal intensity impossible recover low rank matrix',
 'scalable gaussian process inference finite data mean variance guarantees gaussian processes gps offer flexible class priors nonparametric bayesian regression popular gp posterior inference methods typically prohibitively slow lack desirable finite data guarantees quality develop approach scalable approximate gp regression finite data guarantees accuracy pointwise posterior mean variance estimates main contribution novel objective approximate inference nonparametric setting preconditioned fisher pf diverge

In [27]:
predtest_lines

['closed form marginal likelihood gamma poisson matrix factorization present novel understandings gamma poisson gap model probabilistic matrix factorization model count data show gap rewritten free score activation matrix gives us new insights estimation topic dictionary matrix maximum marginal likelihood estimation particular explains robustness estimator specified values factorization rank especially ability automatically prune irrelevant dictionary columns empirically observed previous work marginalization activation matrix leads turn new monte carlo expectation maximization algorithm favorable properties',
 'laboratory mid ir spectra equilibrated igneous meteorites searching observables planetesimal debris meteorites contain minerals solar system asteroids different properties like size presence water core formation provide new mid ir transmission spectra powdered meteorites obtain templates mid ir spectra asteroidal debris would look like essential interpreting mid ir spectra past

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

countvector = CountVectorizer()
X_train_cv = countvector.fit_transform(train_lines)
X_test_cv = countvector.transform(test_lines)

test_cv = countvector.transform(predtest_lines)

In [29]:
#Using TfidfVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

tfidfvector = TfidfTransformer()
X_train_tf = tfidfvector.fit_transform(X_train_cv)
X_test_tf = tfidfvector.fit_transform(X_test_cv)

test_tf = tfidfvector.fit_transform(test_cv)

#VANILLA LOGISTIC REGRESSION:

In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier # Binary Relevance

lr = LogisticRegression(class_weight='balanced',C=3.0)
clf = OneVsRestClassifier(lr)

In [80]:
clf.fit(X_tf, y)

OneVsRestClassifier(estimator=LogisticRegression(C=3.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [81]:
preds = clf.predict(X_test_tf)
preds

array([[0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 0]])

In [82]:
predss = clf.predict(test_tf)
predss

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 1, 1, 0],
       [1, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [83]:
test1 = pd.read_csv('/content/drive/My Drive/Hackathon/Janatahack Independence Day 2020 ML Hackathon/test.csv')

In [84]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predss[:,0],'Physics':predss[:,1],'Mathematics':predss[:,2],'Statistics':predss[:,3],'Quantitative Biology':predss[:,4],'Quantitative Finance':predss[:,5]})

In [85]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,1,0,0,0


In [86]:
submit.to_csv('submissionjhlr00.csv', index=False)

In [70]:
#0.803428979640433
#0.82015145140934 --> full

#SVM:

In [40]:
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

model = SVC()
models = MultiOutputClassifier(model)

In [ ]:
models.fit(X_train_tf, y_train)

In [ ]:
preds = models.predict(X_test_tf)
preds

array([[0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
predssv = models.predict(test_tf)
predssv

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predssv[:,0],'Physics':predssv[:,1],'Mathematics':predssv[:,2],'Statistics':predssv[:,3],'Quantitative Biology':predssv[:,4],'Quantitative Finance':predssv[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,1,0,0,0


In [ ]:
submit.to_csv('submissionjhsvc0.csv', index=False)

In [ ]:
#0.812286689419795

#VANILLA RANDOMFOREST:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, KFold
from sklearn.multioutput import MultiOutputClassifier

clf = MultiOutputClassifier(RandomForestClassifier(random_state=42, class_weight="balanced",n_estimators=500))

In [ ]:
clf.fit(X_train_tf, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight='balanced',
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
            

In [ ]:
preds = clf.predict(X_test_tf)
preds

array([[0, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
predss_rf = clf.predict(test_tf)
predss_rf

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0]])

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predss_rf[:,0],'Physics':predss_rf[:,1],'Mathematics':predss_rf[:,2],'Statistics':predss_rf[:,3],'Quantitative Biology':predss_rf[:,4],'Quantitative Finance':predss_rf[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [ ]:
submit.to_csv('submissionjhRF1.csv', index=False)

In [ ]:
#0.786310517529215

#RANDOMFOREST WITH HYPERPARAMETER TUNING:

###USING RANDOMSEARCHCV

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rff = RandomForestClassifier()
rf = MultiOutputClassifier(rff)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(X_train_tf, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf=RandomForestClassifier()

#put best parameters here

In [ ]:
rf.fit(X_train_tf,y_train)

In [ ]:
preds = rf.predict(X_test_tf)
preds

In [ ]:
predss_rfht = rf.predict(test_tf)
predss_rfht

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predss_rfht[:,0],'Physics':predss_rfht[:,1],'Mathematics':predss_rfht[:,2],'Statistics':predss_rfht[:,3],'Quantitative Biology':predss_rfht[:,4],'Quantitative Finance':predss_rfht[:,5]})

In [ ]:
submit.head()

In [ ]:
submit.to_csv('submissionjhRFHT.csv', index=False)

###USING GRIDSEARCHCV

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
from sklearn.model_selection import GridSearchCV# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True,False],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}# Create a based model
rf = RandomForestClassifier()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search.fit(X_train_tf, y_train)

Fitting 3 folds for each of 576 candidates, totalling 1728 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 31.5min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 49.3min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 70.6min
[Parallel(n_jobs=-1)]: Done 1728 out of 1728 | elapsed: 83.7min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [ ]:
#ABOVE FIT TOOK 90 MINS

In [ ]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 80,
 'max_features': 2,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 100}

In [ ]:
rf=RandomForestClassifier(bootstrap= True, max_depth= 80,max_features= 2,min_samples_leaf= 3,min_samples_split= 8,n_estimators= 100)

#put best parameters here

In [ ]:
rf.fit(X_train_tf,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=80, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
preds = rf.predict(X_test_tf)
preds

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
predss_rfhtg = rf.predict(test_tf)
predss_rfhtg

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
predss_rfhtg[predss_rfhtg[:,0]==1]

array([], shape=(0, 6), dtype=int64)

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predss_rfhtg[:,0],'Physics':predss_rfhtg[:,1],'Mathematics':predss_rfhtg[:,2],'Statistics':predss_rfhtg[:,3],'Quantitative Biology':predss_rfhtg[:,4],'Quantitative Finance':predss_rfhtg[:,5]})

In [ ]:
submit.head()

In [ ]:
submit.to_csv('submissionjhRFHTG.csv', index=False)

#LINEAR SVC:

In [ ]:
from sklearn.svm import LinearSVC, NuSVC

model = LinearSVC()
models = MultiOutputClassifier(model)

In [ ]:
models.fit(X_train_tf, y_train)

MultiOutputClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                          fit_intercept=True,
                                          intercept_scaling=1,
                                          loss='squared_hinge', max_iter=2000,
                                          multi_class='ovr', penalty='l2',
                                          random_state=None, tol=0.0001,
                                          verbose=0),
                      n_jobs=None)

In [ ]:
preds = models.predict(X_test_tf)
preds

array([[0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
predssv = models.predict(test_tf)
predssv

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predssv[:,0],'Physics':predssv[:,1],'Mathematics':predssv[:,2],'Statistics':predssv[:,3],'Quantitative Biology':predssv[:,4],'Quantitative Finance':predssv[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,1,0,0,0


In [ ]:
submit.to_csv('submissionjhlsvc1.csv', index=False)

In [ ]:
#0.810301218670959

#PASSIVE AGRESSIVE CLASSIFIER:

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier

clf = PassiveAggressiveClassifier()
models = MultiOutputClassifier(clf)

In [ ]:
models.fit(X_train_tf, y_train)

MultiOutputClassifier(estimator=PassiveAggressiveClassifier(C=1.0,
                                                            average=False,
                                                            class_weight=None,
                                                            early_stopping=False,
                                                            fit_intercept=True,
                                                            loss='hinge',
                                                            max_iter=1000,
                                                            n_iter_no_change=5,
                                                            n_jobs=None,
                                                            random_state=None,
                                                            shuffle=True,
                                                            tol=0.001,
                                                            validation_fraction=0.1,
      

In [ ]:
preds = models.predict(X_test_tf)
preds

array([[0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0]])

In [ ]:
predspa = models.predict(test_tf)
predspa

array([[0, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predspa[:,0],'Physics':predspa[:,1],'Mathematics':predspa[:,2],'Statistics':predspa[:,3],'Quantitative Biology':predspa[:,4],'Quantitative Finance':predspa[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,1,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,1,0,0,0


In [ ]:
submit.to_csv('submissionjhpa.csv', index=False)

In [ ]:
#0.781271205609591

#SGD CLASSIFIER:

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(epsilon=0.01, max_iter=2000)
multi_target_classifier = MultiOutputClassifier(clf, n_jobs=-1)

In [ ]:
multi_target_classifier.fit(X_train_tf, y_train)

MultiOutputClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                              class_weight=None,
                                              early_stopping=False,
                                              epsilon=0.01, eta0=0.0,
                                              fit_intercept=True, l1_ratio=0.15,
                                              learning_rate='optimal',
                                              loss='hinge', max_iter=2000,
                                              n_iter_no_change=5, n_jobs=None,
                                              penalty='l2', power_t=0.5,
                                              random_state=None, shuffle=True,
                                              tol=0.001,
                                              validation_fraction=0.1,
                                              verbose=0, warm_start=False),
                      n_jobs=-1)

In [ ]:
preds = multi_target_classifier.predict(X_test_tf)
preds

array([[0, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
predssgd = multi_target_classifier.predict(test_tf)
predssgd

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predssgd[:,0],'Physics':predssgd[:,1],'Mathematics':predssgd[:,2],'Statistics':predssgd[:,3],'Quantitative Biology':predssgd[:,4],'Quantitative Finance':predssgd[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,1,0,0,0


In [ ]:
submit.to_csv('submissionjhsgd33.csv', index=False)

In [ ]:
#0.8146244598 
#0.8160465116 changed epsilon=0.01 and max_iter=2000        --->epsilon=0.01 and is penalty=l2 working better

#XGB:

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

clf = XGBClassifier()
model = MultiOutputClassifier(clf)

In [ ]:
model.fit(X_train_tf,y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                              colsample_bylevel=1,
                                              colsample_bynode=1,
                                              colsample_bytree=1, gamma=0,
                                              learning_rate=0.1,
                                              max_delta_step=0, max_depth=3,
                                              min_child_weight=1, missing=None,
                                              n_estimators=100, n_jobs=1,
                                              nthread=None,
                                              objective='binary:logistic',
                                              random_state=0, reg_alpha=0,
                                              reg_lambda=1, scale_pos_weight=1,
                                              seed=None, silent=None,
                                              subsample=

In [ ]:
pred = model.predict(X_test_tf)
pred

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
predsxgb = model.predict(test_tf)
predsxgb

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predsxgb[:,0],'Physics':predsxgb[:,1],'Mathematics':predsxgb[:,2],'Statistics':predsxgb[:,3],'Quantitative Biology':predsxgb[:,4],'Quantitative Finance':predsxgb[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,0,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [ ]:
submit.to_csv('submissionjhxgb.csv', index=False)

In [ ]:
#0.717257027606202

#ADABOOST CLASSIFIER:

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier()
model = MultiOutputClassifier(clf)

In [ ]:
model.fit(X_train_tf,y_train)

MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                   base_estimator=None,
                                                   learning_rate=1.0,
                                                   n_estimators=50,
                                                   random_state=None),
                      n_jobs=None)

In [ ]:
pred = model.predict(X_test_tf)
pred

array([[0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
predsada = model.predict(test_tf)
predsada

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predsada[:,0],'Physics':predsada[:,1],'Mathematics':predsada[:,2],'Statistics':predsada[:,3],'Quantitative Biology':predsada[:,4],'Quantitative Finance':predsada[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,1,0,0,0


In [ ]:
submit.to_csv('submissionjhada.csv', index=False)

In [ ]:
#0.706325118368338

#NAIVE BAYES:

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
model = MultiOutputClassifier(clf)

In [ ]:
model.fit(X_train_tf, y_train)

MultiOutputClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                              fit_prior=True),
                      n_jobs=None)

In [ ]:
pred = model.predict(X_test_tf)
pred

array([[0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [ ]:
prednb = model.predict(test_tf)
prednb

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0]])

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': prednb[:,0],'Physics':prednb[:,1],'Mathematics':prednb[:,2],'Statistics':prednb[:,3],'Quantitative Biology':prednb[:,4],'Quantitative Finance':prednb[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [ ]:
submit.to_csv('submissionjhnb.csv', index=False)

In [ ]:
#0.725109083696946

#MLP CLASSIFIER:

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier()
model = MultiOutputClassifier(clf)

In [ ]:
model.fit(X_train_tf, y_train)

MultiOutputClassifier(estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                              batch_size='auto', beta_1=0.9,
                                              beta_2=0.999,
                                              early_stopping=False,
                                              epsilon=1e-08,
                                              hidden_layer_sizes=(100,),
                                              learning_rate='constant',
                                              learning_rate_init=0.001,
                                              max_fun=15000, max_iter=200,
                                              momentum=0.9, n_iter_no_change=10,
                                              nesterovs_momentum=True,
                                              power_t=0.5, random_state=None,
                                              shuffle=True, solver='adam',
                                              tol=0.0001,


In [ ]:
pred = model.predict(X_test_tf)
pred

array([[0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0]])

In [ ]:
predmlp = model.predict(test_tf)
predmlp

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predmlp[:,0],'Physics':predmlp[:,1],'Mathematics':predmlp[:,2],'Statistics':predmlp[:,3],'Quantitative Biology':predmlp[:,4],'Quantitative Finance':predmlp[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [ ]:
submit.to_csv('submissionjhmlp.csv', index=False)

In [ ]:
#0.794801641586867

#LGBM CLASSIFIER:

In [43]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(class_weight='balanced')
model = MultiOutputClassifier(clf)

In [44]:
model.fit(X_train_tf,y_train)

MultiOutputClassifier(estimator=LGBMClassifier(boosting_type='gbdt',
                                               class_weight='balanced',
                                               colsample_bytree=1.0,
                                               importance_type='split',
                                               learning_rate=0.1, max_depth=-1,
                                               min_child_samples=20,
                                               min_child_weight=0.001,
                                               min_split_gain=0.0,
                                               n_estimators=100, n_jobs=-1,
                                               num_leaves=31, objective=None,
                                               random_state=None, reg_alpha=0.0,
                                               reg_lambda=0.0, silent=True,
                                               subsample=1.0,
                                               subsample_f

In [ ]:
pred = model.predict(X_test_tf)
pred

array([[0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 0]])

In [45]:
predlgb = model.predict(test_tf)
predlgb

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 1, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0]])

In [46]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predlgb[:,0],'Physics':predlgb[:,1],'Mathematics':predlgb[:,2],'Statistics':predlgb[:,3],'Quantitative Biology':predlgb[:,4],'Quantitative Finance':predlgb[:,5]})

In [47]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [48]:
submit.to_csv('submissionjhlgb0.csv', index=False)

In [ ]:
#0.802799128140415

#CATBOOST CLASSIFIER:

In [ ]:
!pip3 install catboost

In [ ]:
import catboost
from catboost import CatBoostClassifier

clf = CatBoostClassifier(learning_rate=0.5, depth= 3, l2_leaf_reg= 40, bootstrap_type= 'Bernoulli', subsample= 0.7,
                         scale_pos_weight= 5, eval_metric= 'AUC', od_type= 'Iter', allow_writing_files= False)
model = MultiOutputClassifier(clf) 

In [ ]:
model.fit(X_train_tf,y_train)

In [ ]:
pred = model.predict(X_test_tf)
pred

array([[0, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 1, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 0]])

In [ ]:
predcb = model.predict(test_tf)
predcb

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 0, 0]])

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predcb[:,0],'Physics':predcb[:,1],'Mathematics':predcb[:,2],'Statistics':predcb[:,3],'Quantitative Biology':predcb[:,4],'Quantitative Finance':predcb[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [ ]:
submit.to_csv('submissionjhcb11.csv', index=False)

In [ ]:
#0.777746131206684 0.1 lr
#0.797897078151855 0.5 lr

#EXTRA TREES CLASSIFIER

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier()
model = MultiOutputClassifier(clf)

In [ ]:
model.fit(X_train_tf,y_train)

MultiOutputClassifier(estimator=ExtraTreesClassifier(bootstrap=False,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                           

In [ ]:
pred = model.predict(X_test_tf)
pred

array([[0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
predet = model.predict(test_tf)
predet

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 0, 0]])

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predet[:,0],'Physics':predet[:,1],'Mathematics':predet[:,2],'Statistics':predet[:,3],'Quantitative Biology':predet[:,4],'Quantitative Finance':predet[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [ ]:
submit.to_csv('submissionjhet.csv', index=False)

In [ ]:
#0.767596937515436

#NEURAL NETWORKS:

In [ ]:
max_len_train = len(max(train_lines,key=len))
max_len_test = len(max(test_lines,key=len))
max_len_predtest = len(max(predtest_lines,key=len))
print(max_len_train, max_len_test, max_len_predtest)

1823 2049 1889


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Flatten, Embedding, BatchNormalization, GRU
from tensorflow.keras.models import Model

In [ ]:
sent_lens_t=[]
for sent in train_lines:
    sent_lens_t.append(len(word_tokenize(sent)))
print('Max length of sent_lens_t', max(sent_lens_t))

Max length of sent_lens_t 284


In [ ]:
print(np.quantile(sent_lens_t,0.98))

185.0


In [ ]:
from keras.preprocessing.text import Tokenizer

max_len_t = 284
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(train_lines)
sequences_train_t = tok.texts_to_sequences(train_lines)

In [ ]:
vocab_len_t=len(tok.index_word.keys())
print('vocab_len_t', vocab_len_t)

vocab_len_t 45764


In [ ]:
sequences_matrix_train_t = pad_sequences(sequences_train_t,padding='pre',maxlen=max_len_train)
print(sequences_matrix_train_t)

[[    0     0     0 ...  1982 11464    30]
 [    0     0     0 ...   298     4  2231]
 [    0     0     0 ...    75   870  2333]
 ...
 [    0     0     0 ...   180  3451    10]
 [    0     0     0 ...  1227  1752  1938]
 [    0     0     0 ...   430   645   471]]


In [ ]:
sequences_test_t = tok.texts_to_sequences(test_lines)
sequences_matrix_test_t = pad_sequences(sequences_test_t,padding='pre',maxlen=max_len_t)
print(sequences_matrix_test_t)

[[   0    0    0 ...   71  365  109]
 [   0    0    0 ...    1  323  938]
 [   0    0    0 ... 1512  294   60]
 ...
 [   0    0    0 ...  392 3364 2362]
 [   0    0    0 ...  175  137  793]
 [   0    0    0 ...  520 1405 2355]]


In [ ]:
print(sequences_matrix_train_t.shape,sequences_matrix_test_t.shape,y1.shape,y2.shape)

(16778, 1823) (4194, 284) (16778, 6) (4194, 6)


In [ ]:
#test:

In [ ]:
sent_lens_ttt=[]
for sent in predtest_lines:
    sent_lens_ttt.append(len(word_tokenize(sent)))
print('Max length of sent_lens_ttt', max(sent_lens_ttt))

Max length of sent_lens_ttt 235


In [ ]:
print(np.quantile(sent_lens_t,0.98))

In [ ]:
max_len_t = 235
sequences_test_ttt = tok.texts_to_sequences(predtest_lines)
sequences_matrix_test_ttt = pad_sequences(sequences_test_ttt,padding='pre',maxlen=max_len_t)
print(sequences_matrix_test_ttt)

[[   0    0    0 ...   21 4532   60]
 [   0    0    0 ...  559 2891 1192]
 [   0    0    0 ... 2791  831  135]
 ...
 [   0    0    0 ...   35 1897 2420]
 [   0    0    0 ...  527 1598 3983]
 [   0    0    0 ...  169  169 4490]]


In [ ]:
print(sequences_matrix_test_ttt.shape)

(8989, 235)


In [ ]:
model = Sequential()
model.add(Embedding(vocab_len_t+1,1000,input_length=max_len_t))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(800,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 235, 1000)         45765000  
_________________________________________________________________
batch_normalization_1 (Batch (None, 235, 1000)         4000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 235000)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 800)               188000800 
_________________________________________________________________
dropout_3 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 500)               400500    
_________________________________________________________________
dropout_4 (Dropout)          (None, 500)              

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
import tensorflow
h = model.fit(sequences_matrix_train_t,y1,
              validation_data=(sequences_matrix_test_t,y2),
              epochs=10, batch_size=64,
              callbacks=[
                         tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=8),
                         tensorflow.keras.callbacks.ModelCheckpoint('/content/sample_data/modeljh_{val_accuracy:.4f}.h5', save_best_only=True,
                                                            save_weights_only=False, monitor='val_accuracy')
    ])

In [ ]:
losses = h.history['loss']
accs = h.history['accuracy']
val_losses = h.history['val_loss']
val_accs = h.history['val_accuracy']
epochs = len(losses)

plt.figure(figsize=(12, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
model = tensorflow.keras.models.load_model('/content/sample_data/modeljh_0.7899.h5')
predss = model.predict(sequences_matrix_test_t)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#USE y_test1 instead of y_test to obtain metrics
print(confusion_matrix(y2, predss))
print(classification_report(y2, predss))
print(accuracy_score(y2, predss))

In [ ]:
#USE y_test1 instead of y_test to obtain metrics
n_right = 0
for i in range(len(predss)):
    if preds[i] == y2[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y2)) * 100)))

In [ ]:
prednn = model.predict()
prednn

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': prednn[:,0],'Physics':prednn[:,1],'Mathematics':prednn[:,2],'Statistics':prednn[:,3],'Quantitative Biology':prednn[:,4],'Quantitative Finance':prednn[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [ ]:
submit.to_csv('submissionjhnb.csv', index=False)

In [ ]:
#

#EASY ENSEMBLE CLASSIFIER:

In [ ]:
!pip install imblearn

In [ ]:
from imblearn.ensemble import EasyEnsembleClassifier

clf = EasyEnsembleClassifier()
model = MultiOutputClassifier(clf)

In [ ]:
model.fit(X_train_tf,y_train)

MultiOutputClassifier(estimator=EasyEnsembleClassifier(base_estimator=None,
                                                       n_estimators=10,
                                                       n_jobs=1,
                                                       random_state=None,
                                                       replacement=False,
                                                       sampling_strategy='auto',
                                                       verbose=0,
                                                       warm_start=False),
                      n_jobs=None)

In [ ]:
pred = model.predict(X_test_tf)
pred

array([[0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 1, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [0, 0, 0, 1, 0, 0],
       [1, 0, 1, 1, 0, 1],
       [0, 0, 1, 1, 0, 0]])

In [ ]:
predeec = model.predict(test_tf)
predeec

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1],
       ...,
       [1, 0, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predeec[:,0],'Physics':predeec[:,1],'Mathematics':predeec[:,2],'Statistics':predeec[:,3],'Quantitative Biology':predeec[:,4],'Quantitative Finance':predeec[:,5]})

In [ ]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,1
3,20976,0,1,0,0,1,0
4,20977,1,0,1,0,0,0


In [ ]:
submit.to_csv('submissionjheec.csv', index=False)

In [ ]:
#0.7111455408153384

#HANDLING IMBALANCE:

In [31]:
train.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,combined
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps ...
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation ...
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,A finite element approximation for the stochas...
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...


#SGD CLASSIFIER FOR FULL DATA:

In [30]:
X.replace('[^a-zA-Z]',' ', regex=True, inplace=True)

for index in X.columns:
  X[index] = X[index].str.lower()

X['combined'] = X['combined'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

X = X.replace('\s+', ' ', regex=True)

X['combined'] = X['combined'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

X_lines = []
for row in range(0,X.shape[0]):
  X_lines.append(' '.join(str(x) for x in X.iloc[row,:]))

In [31]:
X_cv = countvector.transform(X_lines)

X_tf = tfidfvector.fit_transform(X_cv)

In [33]:
from sklearn.linear_model import SGDClassifier
from sklearn.multioutput import MultiOutputClassifier

clf = SGDClassifier(epsilon=0.01, max_iter=2000,class_weight='balanced')
multi_target_classifier = MultiOutputClassifier(clf, n_jobs=-1)

In [34]:
multi_target_classifier.fit(X_tf, y)

MultiOutputClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                              class_weight='balanced',
                                              early_stopping=False,
                                              epsilon=0.01, eta0=0.0,
                                              fit_intercept=True, l1_ratio=0.15,
                                              learning_rate='optimal',
                                              loss='hinge', max_iter=2000,
                                              n_iter_no_change=5, n_jobs=None,
                                              penalty='l2', power_t=0.5,
                                              random_state=None, shuffle=True,
                                              tol=0.001,
                                              validation_fraction=0.1,
                                              verbose=0, warm_start=False),
                      n_jobs=-1)

In [35]:
preds = multi_target_classifier.predict(X_test_tf)
preds

array([[0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 1]])

In [36]:
predssgd = multi_target_classifier.predict(test_tf)
predssgd

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [39]:
submit = pd.DataFrame({'ID': test1.ID, 'Computer Science': predssgd[:,0],'Physics':predssgd[:,1],'Mathematics':predssgd[:,2],'Statistics':predssgd[:,3],'Quantitative Biology':predssgd[:,4],'Quantitative Finance':predssgd[:,5]})

In [40]:
submit.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,1,0,0,0


In [41]:
submit.to_csv('submissionjhsgd00.csv', index=False)

In [42]:
#0.821200377081806